## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,79.07,69,17,3.96,few clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-12.46,69,0,11.10,clear sky
2,2,Bluff,NZ,-46.6000,168.3333,48.99,78,75,13.00,broken clouds
3,3,Catuday,PH,16.2923,119.8062,76.86,80,13,9.62,few clouds
4,4,Ljungby,SE,56.8332,13.9408,26.60,93,75,11.50,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,79.07,69,17,3.96,few clouds
3,3,Catuday,PH,16.2923,119.8062,76.86,80,13,9.62,few clouds
8,8,Acarau,BR,-2.8856,-40.1200,87.66,58,100,16.17,overcast clouds
15,15,Vaini,TO,-21.2000,-175.2000,75.20,88,90,25.32,light rain
18,18,Hambantota,LK,6.1241,81.1185,77.52,82,76,3.36,broken clouds
19,19,Yadgir,IN,16.7667,77.1333,75.04,44,26,11.95,scattered clouds
23,23,Carnarvon,AU,-24.8667,113.6333,75.20,78,0,21.85,clear sky
32,32,Atuona,PF,-9.8000,-139.0333,78.17,77,6,18.59,clear sky
35,35,Rio Grande,BR,-32.0350,-52.0986,84.20,65,40,11.50,scattered clouds
38,38,Hithadhoo,MV,-0.6000,73.0833,81.50,72,100,9.73,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                220
City                   220
Country                218
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()



City_ID                220
City                   220
Country                218
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [7]:
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None 
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.07,few clouds,-23.1203,-134.9692,None
3,Catuday,PH,76.86,few clouds,16.2923,119.8062,None
8,Acarau,BR,87.66,overcast clouds,-2.8856,-40.1200,None
15,Vaini,TO,75.20,light rain,-21.2000,-175.2000,None
18,Hambantota,LK,77.52,broken clouds,6.1241,81.1185,None
19,Yadgir,IN,75.04,scattered clouds,16.7667,77.1333,None
23,Carnarvon,AU,75.20,clear sky,-24.8667,113.6333,None
32,Atuona,PF,78.17,clear sky,-9.8000,-139.0333,None
35,Rio Grande,BR,84.20,scattered clouds,-32.0350,-52.0986,None
38,Hithadhoo,MV,81.50,overcast clouds,-0.6000,73.0833,None


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    


    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.


In [10]:
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
660,Gao,ML,86.27,clear sky,16.6362,1.6370,None
661,Vao,NC,77.02,scattered clouds,-22.6667,167.4833,None
663,Vicosa Do Ceara,BR,81.03,overcast clouds,-3.5622,-41.0922,None
664,Alcantara,BR,80.60,light rain,-2.4089,-44.4147,None
673,Necochea,AR,79.00,scattered clouds,-38.5473,-58.7368,None
675,Juruti,BR,87.10,overcast clouds,-2.1522,-56.0922,None
676,Miramar,US,75.20,scattered clouds,25.9873,-80.2323,None
678,Santa Rosa De Aguan,HN,84.20,few clouds,15.9500,-85.7167,None
687,Barra Patuca,HN,79.39,scattered clouds,15.8000,-84.2833,None
688,Sagua De Tanamo,CU,84.27,clear sky,20.5819,-75.2414,None


In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("Nan")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)



In [12]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11alocations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.